In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
import mediapipe as mp
from moviepy.editor import VideoFileClip

# YOLOv8 모델 로드 (YOLOv8s 모델을 예제로 사용)
model = YOLO('yolov8s.pt')

# MediaPipe 포즈 추정기 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 비디오 파일 경로와 타임스탬프 정보를 딕셔너리 형태로 저장
directories = {
    'broken': '/content/drive/MyDrive/DL/output_all/data_2/파손'
}

# 결과를 저장할 경로 설정
output_base_dir = '/content/drive/MyDrive/DL/output_all/mediapipe/processed_videos'
csv_output_base_dir = '/content/drive/MyDrive/DL/output_all/mediapipe/processed_csvs'

# 저장할 디렉토리가 없으면 생성
os.makedirs(output_base_dir, exist_ok=True)
os.makedirs(csv_output_base_dir, exist_ok=True)

# CSV 결과를 저장할 리스트 초기화
results_list = []

def process_frame(frame, boxes):
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        cls = box.cls[0].cpu().numpy()
        label = model.names[int(cls)]

        if label == 'person':
            person_crop = frame[int(y1):int(y2), int(x1):int(x2)]
            person_rgb = cv2.cvtColor(person_crop, cv2.COLOR_BGR2RGB)
            results_pose = pose.process(person_rgb)

            if results_pose.pose_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(person_crop, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                # 랜드마크 좌표 저장
                landmarks = results_pose.pose_landmarks.landmark
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # 바운딩 박스 그리기
                return [(lm.x, lm.y, lm.z, lm.visibility) for lm in landmarks]  # 랜드마크 추가
    return []

for behavior, directory in directories.items():
    print(f"Processing {behavior} videos...")

    for filename in os.listdir(directory):
        video_filepath = os.path.join(directory, filename)

        # 확장자가 비디오 파일이 아닌 경우 건너뛰기
        if not video_filepath.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
            continue

        output_video_path = os.path.join(output_base_dir, f"processed_{filename}")
        csv_path = os.path.join(csv_output_base_dir, f"{filename}.csv")

        try:
            # 비디오 파일 로드
            clip = VideoFileClip(video_filepath)

            # 비디오 라이터 초기화
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, clip.fps, (int(clip.w), int(clip.h)))

            # 현재 비디오 파일에 대한 결과를 저장할 리스트 초기화
            current_video_results = []

            # 클립에서 프레임 단위로 처리
            frame_count = 0
            for frame in clip.iter_frames(fps=clip.fps):
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                results = model(frame_bgr)

                # YOLO 모델이 제대로 객체를 감지하는지 확인
                print(f"Frame {frame_count}: Detected {len(results[0].boxes)} objects")

                landmarks = process_frame(frame_bgr, results[0].boxes)
                if landmarks:
                    row = {
                        'filename': filename,
                        'frame': frame_count,
                        'behavior': behavior,
                    }
                    for i, (x, y, z, vis) in enumerate(landmarks):
                        row[f'landmark_{i}_x'] = x
                        row[f'landmark_{i}_y'] = y
                        row[f'landmark_{i}_z'] = z
                        row[f'landmark_{i}_vis'] = vis
                    current_video_results.append(row)

                    cv2.putText(frame_bgr, behavior, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # 결과를 비디오에 작성
                out.write(frame_bgr)
                frame_count += 1

            # 현재 비디오 파일에 대한 결과를 전체 결과 리스트에 추가
            results_list.extend(current_video_results)

            # CSV 파일 생성 (현재 비디오 파일에 대한 결과)
            results_df = pd.DataFrame(current_video_results)
            results_df.to_csv(csv_path, index=False)

            # 리소스 해제
            out.release()
            clip.reader.close()
            if clip.audio is not None:
                clip.audio.reader.close_proc()

        except Exception as e:
            print(f"Error processing file {video_filepath}: {e}")

print("모든 클립 처리 완료!")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0: 384x640 1 person, 1 bottle, 1 donut, 1 tv, 26.2ms
Speed: 3.4ms preprocess, 26.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Frame 76: Detected 4 objects

0: 384x640 1 person, 1 bottle, 1 tv, 18.0ms
Speed: 3.3ms preprocess, 18.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Frame 77: Detected 3 objects

0: 384x640 1 person, 2 bottles, 1 tv, 13.9ms
Speed: 4.7ms preprocess, 13.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Frame 78: Detected 4 objects

0: 384x640 1 person, 1 bottle, 1 dining table, 1 tv, 21.9ms
Speed: 5.7ms preprocess, 21.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Frame 79: Detected 4 objects

0: 384x640 1 person, 1 bottle, 1 donut, 1 dining table, 1 tv, 16.4ms
Speed: 3.4ms preprocess, 16.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Frame 80: Detected 5 objects

0: 384x640 1 person, 1 donut, 1 tv, 20.4ms
Speed: 3.7ms pr